<a href="https://colab.research.google.com/github/mabittar/Learn/blob/master/Covid_Counter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [72]:
#to use Covid Counter from https://github.com/sudharshan-ashok/pycovid
!pip install pycovid


In [0]:
#import modules
import pandas as pd
from datetime import datetime, date
import re
import numpy as np
import pandas as pd

In [0]:
#import country population database
population_url = "https://raw.githubusercontent.com/julianbruegger/corona-display/master/data/population.csv"
colnames_remap= {"Country":"country",
                 "Country Code":"alpha-3",
                 "2018":"population" 
                }
col_drop = ["country"]
population_df = pd.read_csv(population_url, encoding= 'unicode_escape', sep=";").rename(columns=colnames_remap)

#corrrect alpha-3 for Iran
population_df.loc[(population_df.country == "Iran"), 'alpha-3'] = 'IRN'
population_df = population_df.drop(columns=col_drop)

#population_df.head(10)
#print(population_df.columns)
#print(population_df.loc[population_df['alpha-3'] == 'IRN'])

In [0]:
#import database for Covid cases
from pycovid import pycovid
df = pycovid.getCovidCases(cumsum=False)
#df.head()


#rename and reorganize columns
renamed_columns_map = {
    "country_region": "country",
    "province_state": "location",
    "lat": "latitude",
    "long": "longitude",
    "type": "status"
    }

cols_to_drop = ["location", "latitude", "longitude", "name"]

df = df.rename(columns=renamed_columns_map).drop(columns=cols_to_drop)
#df.head(10)

In [0]:
#define important dates
last_date = df['date'].max()
last_date = np.datetime64(last_date, 'D')
#print(last_date)
#print(last_date_str)


today = date.today()
#print(today)

In [0]:
df = df.merge(population_df, on='alpha-3', how='left')  # Merge df (covid_cases) with population_df


In [0]:
# Create a function that takes inputs, cases and population to calculare % infected
def pop_cases(cases, population):
    # returns the % between cases and population
    return cases / population

# Create a variable that is the output of the function
df['percentage'] = pop_cases(df['cases'], df['population'])

#df.head(10)

In [0]:
#df['status'].value_counts()

In [0]:
confirmed_cases_df = df[df.status.eq('confirmed')]
#confirmed_cases_df.head()

In [0]:
#Group country with more confirmed cases
high_cases_df = confirmed_cases_df.groupby(['alpha-3','country']).sum().reset_index()
high_cases_df = high_cases_df.sort_values(by='cases', ascending=False).head(10)
total_cases = high_cases_df['cases'].sum(axis = 0, skipna = True) 
#high_cases_df.head(10)

#Group country with biggest population % infected
high_perct_cases_df = high_cases_df.sort_values(by='percentage', ascending=False).head(10)
#high_perct_cases_df

In [0]:
#filter biggest increase cases in last date of dataframe
increase_df = confirmed_cases_df[confirmed_cases_df['date'].eq(last_date)].sort_values(by='cases', ascending=False).head(10)
#increase_df.head()

In [0]:
total_increase = increase_df['cases'].sum(axis = 0, skipna = True) 
#print(total_increase)

In [0]:
#import database for Covid recovered cases
recovered_cases_df = df[df.status.eq('recovered')]
#recovered_cases_df.head()

#Group countries with more recovered cases
high_recovered_df = recovered_cases_df.groupby(['alpha-3','country']).sum().reset_index()
high_recovered_df = high_recovered_df.sort_values(by='cases', ascending=False).head(10)
total_recovered = recovered_cases_df['cases'].sum(axis = 0, skipna = True) 
#high_recovered_df.tail(10)
#print(total_recovered)

In [0]:
#import database for Covid death cases
death_cases_df = df[df.status.eq('death')]
#recovered_cases_df.head()

#Group countries with more deaths cases
high_death_cases_df = death_cases_df.groupby(['alpha-3','country']).sum().reset_index()
high_death_cases_df = high_death_cases_df.sort_values(by='cases', ascending=False).head(10)
total_death = high_death_cases_df['cases'].sum(axis = 0, skipna = True) 
#high_death_cases_df.head(10)

#filter increase death cases in last date of dataframe
increase_death_df = death_cases_df[death_cases_df['date'].eq(last_date)].sort_values(by='cases', ascending=False).head(10)
#increase_death_df.head()


In [103]:
perct_recovered = total_recovered / total_cases
print(perct_recovered)

0.15524682571314696


In [108]:
#Display countries in number of confirmed cases
country1 = high_cases_df['country'].iloc[0]
cases1 = high_cases_df['cases'].iloc[0]
percent_pop1 = high_cases_df['percentage'].iloc[0]
country2 = high_cases_df['country'].iloc[1]
cases2 = high_cases_df['cases'].iloc[1]
percent_pop2 = high_cases_df['percentage'].iloc[1]
country3 = high_cases_df['country'].iloc[2]
cases3 = high_cases_df['cases'].iloc[2]
percent_pop3 = high_cases_df['percentage'].iloc[2]

#Display increased cases
increase_country = increase_df['country'].iloc[0]
new_cases = increase_df['cases'].iloc[0]

#Display country with most recovered cases
country_recover = high_recovered_df['country'].iloc[0]
cases_recover = high_recovered_df['cases'].iloc[0]
perct_recovered = total_recovered / total_cases

#Display countries with most deaths
country_death = high_death_cases_df['country'].iloc[0]
cases_death = high_death_cases_df['cases'].iloc[0]


popupmsg = ('              Report Covid-19 cases in {}:'
        '\n{} is the country with more recovered with {:,} cases reported. The worldwide total report of recovered cases is {:,} until now.'
        '\n  The percentage of recoverd cases is {:.2%}.'
        '\n\n               Countries with more confirmed cases:'
        '\n  1: {} with {:,} confirmed cases. It´s represent {:.3%} of {} population.'
        '\n  2: {} with {:,} confirmed cases. It´s represents {:.3%} of {} population.'
        '\n  3: {} with {:,} confirmed cases. It´s represents {:.3%} of {} population.'
        '\n\nThe country with biggest increase is {} with {:,} new cases.'
        '\nThe total of new reported cases is {:,}. The total confirmed cases is {:,}.'
        '\n\n{} is the country with more death reported with {:,}. Worldwide deaths reported by covid is {:,}.'
        .format(
            last_date,
            country_recover, cases_recover, total_recovered,
            perct_recovered,
            
            country1,cases1,percent_pop1,country1,
            country2,cases2,percent_pop2,country2,
            country3,cases3,percent_pop3,country3,
            increase_country,new_cases,
            total_increase,total_cases,
            country_death, cases_death, total_death)
        )

print(popupmsg)

              Report Covid-19 cases in 2020-03-31:
Spain is the country with more recovered with 19,259 cases reported. The worldwide total report of recovered cases is 95,431 until now.
  The percentage of recoverd cases is 15.52%.

               Countries with more confirmed cases:
  1: US with 188,172 confirmed cases. It´s represent 0.058% of US population.
  2: Italy with 105,792 confirmed cases. It´s represents 0.175% of Italy population.
  3: Spain with 95,923 confirmed cases. It´s represents 0.205% of Spain population.

The country with biggest increase is US with 26,365 new cases.
The total of new reported cases is 59,749. The total confirmed cases is 614,705.

Italy is the country with more death reported with 12,428. Worldwide deaths reported by covid is 34,362.


In [0]:
#import tkinter as tk
#def popupmsg(msg):
#     gui = tk.Toplevel()
#     gui.title("Covid Cases")
#     label = tk.Label(gui, text=msg) #Can add a font arg here
#     label.pack(side="top", fill="x", pady=10)
#     B1 = tk.Button(gui, text="Okay", command = gui.destroy)
#     B1.pack()
#     gui.mainloop()

#popupmsg (pop_up_msg)

